In [29]:
import csv
import os
from matplotlib import pyplot as plt
import cv2
import numpy as np
import random

USE_CV2_IMREAD = True


class DataLoader:
    
    def __init__(
            self, data_file='data/driving_log.csv',
            batch_size=32):
        self.batch_size = batch_size
        with open(data_file) as inf:
            self.csv_lines = list(csv.reader(inf))
            random.shuffle(self.csv_lines)

    @staticmethod
    def imread(path):
        if USE_CV2_IMREAD:
            imdata = cv2.imread(path)
        else:
            imdata = plt.imread(path)
            imdata = np.cast(np.round(imdata * 255), np.uint8)
        
        return imdata

    def load_data(self):
        raise Exception('bitrotted')
        batch_x = []
        batch_y = []
        for line in self.csv_lines:
            forward_img_path = self.fix_img_path(line[0])
            imdata = self.imread(forward_img_path)
            
            batch_x.append(imdata)
            batch_y.append(float(line[3]))
            
            if len(batch_x) == BATCH_SIZE:
                yield np.array(batch_x), np.array(batch_y)
                batch_x, batch_y = [], []
        
    def num_batches(self):
        return len(self.csv_lines) // BATCH_SIZE
        
    @staticmethod
    def fix_img_path(path):
        fname = path.split('/')[-1]
        return os.path.join(os.getcwd(), 'data', 'IMG', fname)
    
    @staticmethod
    def all_img_paths(path):
        center = fix_img_path(path)
        left = 'left'.join(center.rsplit('center', 1))
        right = 'right'.join(center.rsplit('right', 1))
        
        return center, left, right
    
    def load_all_data(self):
        rv_x = []
        rv_y = []
        for line in self.csv_lines:
            paths = map(self.fix_img_path, line[:3])  # center, left, right
            offsets = [0, 0.2, -0.2]  # correct
            for path, offset in zip(paths, offsets):
                imdata = self.imread(path)
                angle = float(line[3]) + offset
                
                rv_x.append(imdata)
                rv_y.append(angle)
                # also append reversed data
                rv_x.append(imdata[:,::-1,:])
                rv_y.append(-angle)
        
        return np.array(rv_x), np.array(rv_y)

# def load_data():
#     with open('data/driving_log.csv') as inf:
#         batch = []
#         reader = csv.reader(inf)
#         for line in reader:
#             forward_img_path = fix_img_path(line[0])
#             imdata = imread(forward_img_path) * 255
#             batch.append((
#                 imdata,
#                 float(line[3])))
            
#             if len(batch) == BATCH_SIZE:
#                 yield batch
#                 batch = []

# forward_cam_data_x = np.array(forward_cam_data_x)
# forward_cam_data_y = np.array(forward_cam_data_y)

# print('{} samples'.format(len(forward_cam_data_x)))

In [30]:
BATCH_SIZE = 32

data_loader = DataLoader(batch_size=BATCH_SIZE)

In [31]:
from keras import models, layers

EPOCHS = 3
USE_GENERATOR_LOADER = False
USE_DEEP_MODEL = True


model = models.Sequential()
model.add(layers.Cropping2D(cropping=((70, 25), (0, 0)), input_shape=(160, 320, 3)))

if USE_DEEP_MODEL:
    model.add(layers.Lambda(lambda x: (x - 128.0) / 2.0))
    model.add(layers.Conv2D(6, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(16, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(84))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1))
else:
    model.add(layers.Flatten())
    model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')

if USE_GENERATOR_LOADER:
    model.fit_generator(data_loader.load_data(),
                        data_loader.num_batches(),
                        epochs=EPOCHS)
else:
    x, y = data_loader.load_all_data()
    model.fit(x, y, validation_split=0.2, epochs=EPOCHS, shuffle=True)

model.save('model1.h5')

Train on 25953 samples, validate on 6489 samples
Epoch 1/3
25953/25953 [==============================] - 24s 907us/step - loss: 32.9403 - val_loss: 0.0416
Epoch 2/3
25953/25953 [==============================] - 23s 870us/step - loss: 0.0441 - val_loss: 0.0416
Epoch 3/3
25953/25953 [==============================] - 23s 874us/step - loss: 0.0430 - val_loss: 0.0416


In [28]:
model.save('model1.h5')

In [5]:
x, y = data_loader.load_all_data()

In [6]:
%matplotlib inline
from matplotlib import pyplot as plt

#plt.imshow(x[0] / 255)